In [1]:
from modularity_aware_gae.evaluation import community_detection, link_prediction
from modularity_aware_gae.input_data import load_data, load_labels
from modularity_aware_gae.louvain import louvain_clustering
from modularity_aware_gae.model import  GCNModelAE, GCNModelVAE, LinearModelAE, LinearModelVAE
from modularity_aware_gae.optimizer import OptimizerAE, OptimizerVAE
from modularity_aware_gae.preprocessing import *
from modularity_aware_gae.sampling import get_distribution, node_sampling
import numpy as np
import os
import scipy.sparse as sp
import tensorflow as tf
import time

tf.app.flags.DEFINE_string('f', '', 'kernel')#添加的，不报错
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS

# Parameters related to the input data
flags.DEFINE_string('dataset', 'celegans', 'Graph dataset, among: cora, citeseer, wiki, celegans, email, polbooks, texas, wisconsin')

flags.DEFINE_boolean('features', False, 'Whether to include node features')


# Parameters related to the Modularity-Aware GAE/VGAE model to train

# 1/3 - General parameters associated with GAE/VGAE
flags.DEFINE_string('model', 'linear_vae', 'Model to train, among: gcn_ae, gcn_vae, \
                                            linear_ae, linear_vae')
flags.DEFINE_float('dropout', 0., 'Dropout rate')
flags.DEFINE_integer('iterations', 200, 'Number of iterations in training')
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate (Adam)')
flags.DEFINE_integer('hidden', 32, 'Dimension of the GCN hidden layer')
flags.DEFINE_integer('dimension', 16, 'Dimension of the output layer, i.e., \
                                       dimension of the embedding space')

# 2/3 - Additional parameters, specific to Modularity-Aware models
flags.DEFINE_float('beta', 0.0, 'Beta hyperparameter of Mod.-Aware models')
flags.DEFINE_float('lamb', 0.0, 'Lambda hyperparameter of Mod.-Aware models')
flags.DEFINE_float('gamma', 1.0, 'Gamma hyperparameter of Mod.-Aware models')
flags.DEFINE_integer('s_reg', 2, 's hyperparameter of Mod.-Aware models')

# 3/3 - Additional parameters, aiming to improve scalability
flags.DEFINE_boolean('fastgae', False, 'Whether to use the FastGAE framework')
flags.DEFINE_integer('nb_node_samples', 1000, 'In FastGAE, number of nodes to \
                                               sample at each iteration, i.e., \
                                               size of decoded subgraph')
flags.DEFINE_string('measure', 'degree', 'In FastGAE, node importance measure used \
                                          for sampling: degree, core, or uniform')
flags.DEFINE_float('alpha', 1.0, 'alpha hyperparameter associated with the FastGAE sampling')
flags.DEFINE_boolean('replace', False, 'Sample nodes with or without replacement')
flags.DEFINE_boolean('simple', True, 'Use simpler (and faster) modularity in optimizers')


# Parameters related to the experimental setup
flags.DEFINE_string('task', 'task_2', 'task_1: pure community detection \
                                       task_2: joint link prediction and \
                                               community detection')
flags.DEFINE_integer('nb_run', 10, 'Number of model run + test')
flags.DEFINE_float('prop_val', 1., 'Proportion of edges in validation set \
                                    for the link prediction task')
flags.DEFINE_float('prop_test', 10., 'Proportion of edges in test set \
                                      for the link prediction task')
flags.DEFINE_boolean('validation', False, 'Whether to compute validation \
                                           results at each iteration, for \
                                           the link prediction task')
flags.DEFINE_boolean('verbose', True, 'Whether to print all comments details')


# Introductory message
if FLAGS.verbose:
    introductory_message()


 
 
 
[MODULARITY-AWARE GRAPH AUTOENCODERS]
 
 
 

EXPERIMENTAL SETTING 

- Graph dataset: celegans
- Mode name: linear_vae
- Number of models to train: 10
- Number of training iterations for each model: 200
- Learning rate: 0.01
- Dropout rate: 0.0
- Use of node features in the input layer: False
- Dimension of the output layer: 16
- lambda: 0.0
- beta: 0.0
- gamma: 1.0
- s: 2
- FastGAE: no 

Final embedding vectors will be evaluated on:
- Task 2, i.e., joint community detection and link prediction

 
 
 



In [3]:
# Initialize lists to collect final results
mean_ami = []
mean_ari = []

mean_roc = []
mean_ap = []

# Load data
if FLAGS.verbose:
    print("LOADING DATA\n")
    print("Loading the", FLAGS.dataset, "graph")
adj_init, features_init = load_data(FLAGS.dataset)
#labels = load_labels(adj_init.shape[0])
if FLAGS.verbose:
    print("- Number of nodes:", adj_init.shape[0])
    #print("- Number of communities:", len(np.unique(labels)))
    print("- Use of node features:", FLAGS.features)
    print("Done! \n \n \n \n")

LOADING DATA

Loading the celegans graph
- Number of nodes: 297
- Use of node features: False
Done! 
 
 
 



In [4]:
# We repeat the entire training+test process FLAGS.nb_run times
for i in range(FLAGS.nb_run):
    if FLAGS.verbose:
        print("EXPERIMENTS ON MODEL", i + 1, "/", FLAGS.nb_run, "\n")
        print("STEP 1/3 - PREPROCESSING STEPS \n")


    # Edge masking for Link Prediction:
    if FLAGS.task == 'task_2':
        # Compute Train/Validation/Test sets
        if FLAGS.verbose:
            print("Masking some edges from the training graph, for link prediction")
            print("(validation set:", FLAGS.prop_val, "% of edges - test set:",
                  FLAGS.prop_test, "% of edges)")
        #adj, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj_init, FLAGS.prop_test, FLAGS.prop_val)
        adj, test_edges, test_edges_false = mask_test_edges(adj_init, FLAGS.prop_test)
        if FLAGS.verbose:
            print("Done! \n")
    else:
        adj = adj_init

    # Compute the number of nodes
    num_nodes = adj.shape[0]

    # Preprocessing on node features
    if FLAGS.verbose:
        print("Preprocessing node features")
    if FLAGS.features:
        features = features_init
    else:
        # If features are not used, replace feature matrix by identity matrix
        features = sp.identity(num_nodes)
    features = sparse_to_tuple(features)
    num_features = features[2][1]
    features_nonzero = features[1].shape[0]
    if FLAGS.verbose:
        print("Done! \n")


    # Community detection using Louvain, as a preprocessing step
    if FLAGS.verbose:
        print("Running the Louvain algorithm for community detection")
        print("as a preprocessing step for the encoder")
    # Get binary community matrix (adj_louvain_init[i,j] = 1 if nodes i and j are
    # in the same community) as well as number of communities found by Louvain
    adj_louvain_init, nb_communities_louvain, partition = louvain_clustering(adj, FLAGS.s_reg)
    if FLAGS.verbose:
        print("Done! Louvain has found", nb_communities_louvain, "communities \n")

    # FastGAE: node sampling for stochastic subgraph decoding - used when facing scalability issues
    if FLAGS.fastgae:
        if FLAGS.verbose:
            print("Preprocessing operations related to the FastGAE method")
        # Node-level p_i degree-based, core-based or uniform distribution
        node_distribution = get_distribution(FLAGS.measure, FLAGS.alpha, adj)
        # Node sampling for initializations
        sampled_nodes, adj_label, adj_sampled_sparse = node_sampling(adj, node_distribution, FLAGS.nb_node_samples, FLAGS.replace)
        if FLAGS.verbose:
            print("Done! \n")
    else:
        sampled_nodes = np.array(range(FLAGS.nb_node_samples))


    # Placeholders
    if FLAGS.verbose:
        print('Setting up the model and the optimizer')
    placeholders = {
        'features': tf.sparse_placeholder(tf.float32),
        'adj': tf.sparse_placeholder(tf.float32),
        'adj_layer2': tf.sparse_placeholder(tf.float32), # Only used for 2-layer GCN encoders
        'degree_matrix': tf.sparse_placeholder(tf.float32),
        'adj_orig': tf.sparse_placeholder(tf.float32),
        'dropout': tf.placeholder_with_default(0., shape = ()),
        'sampled_nodes': tf.placeholder_with_default(sampled_nodes, shape = [FLAGS.nb_node_samples])
    }


    # Create model
    if FLAGS.model == 'linear_ae':
        # Linear Graph Autoencoder
        model = LinearModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'linear_vae':
        # Linear Graph Variational Autoencoder
        model = LinearModelVAE(placeholders, num_features, num_nodes, features_nonzero)
    elif FLAGS.model == 'gcn_ae':
        # 2-layer GCN Graph Autoencoder
        model = GCNModelAE(placeholders, num_features, features_nonzero)
    elif FLAGS.model == 'gcn_vae':
        # 2-layer GCN Graph Variational Autoencoder
        model = GCNModelVAE(placeholders, num_features, num_nodes, features_nonzero)
    else:
        raise ValueError('Undefined model!')


    # Optimizer
    if FLAGS.fastgae:
        num_sampled = adj_sampled_sparse.shape[0]
        sum_sampled = adj_sampled_sparse.sum()
        pos_weight = float(num_sampled * num_sampled - sum_sampled) / sum_sampled
        norm = num_sampled * num_sampled / float((num_sampled * num_sampled - sum_sampled) * 2)
    else:
        pos_weight = float(num_nodes * num_nodes - adj.sum()) / adj.sum()
        norm = num_nodes * num_nodes / float((num_nodes * num_nodes - adj.sum()) * 2)

    if FLAGS.model in ('gcn_ae', 'linear_ae'):
        opt = OptimizerAE(preds = model.reconstructions,
                          labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'], validate_indices = False), [-1]),
                          degree_matrix = tf.reshape(tf.sparse_tensor_to_dense(placeholders['degree_matrix'], validate_indices = False), [-1]),
                          num_nodes = num_nodes,
                          pos_weight = pos_weight,
                          norm = norm,
                          clusters_distance = model.clusters)
    else:
        opt = OptimizerVAE(preds = model.reconstructions,
                           labels = tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'], validate_indices = False), [-1]),
                           degree_matrix = tf.reshape(tf.sparse_tensor_to_dense(placeholders['degree_matrix'], validate_indices = False), [-1]),
                           model = model,
                           num_nodes = num_nodes,
                           pos_weight = pos_weight,
                           norm = norm,
                           clusters_distance = model.clusters)
    if FLAGS.verbose:
        print("Done! \n")


    # Symmetrically normalized "message passing" matrices
    if FLAGS.verbose:
        print("Preprocessing on message passing matrices")
    adj_norm = preprocess_graph(adj + FLAGS.lamb*adj_louvain_init)
    adj_norm_layer2 = preprocess_graph(adj)
    if not FLAGS.fastgae:
        adj_label = sparse_to_tuple(adj + sp.eye(num_nodes))
    if FLAGS.verbose:
        print("Done! \n")


    # Degree matrices
    deg_matrix, deg_matrix_init = preprocess_degree(adj, FLAGS.simple)


    # Initialize TF session
    if FLAGS.verbose:
        print("Initializing TF session")
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    if FLAGS.verbose:
        print("Done! \n")


    # Model training
    if FLAGS.verbose:
        print("STEP 2/3 - MODEL TRAINING \n")
        print("Starting training")

    for iter in range(FLAGS.iterations):

        # Flag to compute running time for each iteration
        t = time.time()

        # Construct feed dictionary
        feed_dict = construct_feed_dict(adj_norm, adj_norm_layer2, adj_label, features, deg_matrix, placeholders)
        if FLAGS.fastgae:
            # Update sampled subgraph and sampled Louvain matrix
            feed_dict.update({placeholders['sampled_nodes']: sampled_nodes})
            # New node sampling
            sampled_nodes, adj_label, adj_label_sparse, = node_sampling(adj, node_distribution, FLAGS.nb_node_samples, FLAGS.replace)

        # Weights update
        outs = sess.run([opt.opt_op, opt.cost, opt.cost_adj, opt.cost_mod],
                        feed_dict = feed_dict)

        # Compute average loss
        avg_cost = outs[1]
        if FLAGS.verbose:
            # Display information on the iteration
            print("Iteration:", '%04d' % (iter + 1), "Loss:", "{:.5f}".format(avg_cost),
                  "Time:", "{:.5f}".format(time.time() - t))

    if FLAGS.verbose:
        print("Done! \n")

    # Compute embedding vectors, for evaluation
    if FLAGS.verbose:
        print("STEP 3/3 - MODEL EVALUATION \n")
        print("Computing the final embedding vectors, for evaluation")
    emb = sess.run(model.z_mean, feed_dict = feed_dict)
    if FLAGS.verbose:
        print("Done! \n")

    # Test model: link prediction (classification edges/non-edges)

    if FLAGS.verbose:
        print("Testing: link prediction")
    # Get ROC and AP scores
    roc_score, ap_score = link_prediction(test_edges, test_edges_false, emb)
    mean_roc.append(roc_score)
    mean_ap.append(ap_score)
    if FLAGS.verbose:
        print("Done! \n")

EXPERIMENTS ON MODEL 1 / 10 

STEP 1/3 - PREPROCESSING STEPS 

Masking some edges from the training graph, for link prediction
(validation set: 1.0 % of edges - test set: 10.0 % of edges)
Done! 

Preprocessing node features
Done! 

Running the Louvain algorithm for community detection
as a preprocessing step for the encoder
Done! Louvain has found 6 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TRAINING 

Starting training
Iteration: 0001 Loss: 1.69279 Time: 0.10698
Iteration: 0002 Loss: 1.63475 Time: 0.00798
Iteration: 0003 Loss: 1.64568 Time: 0.00701
Iteration: 0004 Loss: 1.68335 Time: 0.00700
Iteration: 0005 Loss: 1.64407 Time: 0.00602
Iteration: 0006 Loss: 1.65096 Time: 0.00598
Iteration: 0007 Loss: 1.59208 Time: 0.00602
Iteration: 0008 Loss: 1.54860 Time: 0.00596
Iteration: 0009 Loss: 1.55702 Time: 0.00764
Iteration: 0010 Loss: 1.49890 Time: 0.00678
Iteration: 0011 Los

Iteration: 0181 Loss: 0.66212 Time: 0.00592
Iteration: 0182 Loss: 0.67430 Time: 0.00660
Iteration: 0183 Loss: 0.67469 Time: 0.00548
Iteration: 0184 Loss: 0.67004 Time: 0.00538
Iteration: 0185 Loss: 0.67636 Time: 0.00661
Iteration: 0186 Loss: 0.66593 Time: 0.00505
Iteration: 0187 Loss: 0.65492 Time: 0.00560
Iteration: 0188 Loss: 0.65893 Time: 0.00609
Iteration: 0189 Loss: 0.65589 Time: 0.00571
Iteration: 0190 Loss: 0.65625 Time: 0.00634
Iteration: 0191 Loss: 0.66073 Time: 0.00500
Iteration: 0192 Loss: 0.66616 Time: 0.00604
Iteration: 0193 Loss: 0.65335 Time: 0.00497
Iteration: 0194 Loss: 0.65130 Time: 0.00592
Iteration: 0195 Loss: 0.65868 Time: 0.00596
Iteration: 0196 Loss: 0.66045 Time: 0.00542
Iteration: 0197 Loss: 0.65039 Time: 0.00608
Iteration: 0198 Loss: 0.64519 Time: 0.00601
Iteration: 0199 Loss: 0.65418 Time: 0.00388
Iteration: 0200 Loss: 0.64544 Time: 0.00696
Done! 

STEP 3/3 - MODEL EVALUATION 

Computing the final embedding vectors, for evaluation
Done! 

Testing: link predic

Iteration: 0154 Loss: 0.69189 Time: 0.00655
Iteration: 0155 Loss: 0.69014 Time: 0.00594
Iteration: 0156 Loss: 0.69831 Time: 0.00704
Iteration: 0157 Loss: 0.68835 Time: 0.00649
Iteration: 0158 Loss: 0.69011 Time: 0.00496
Iteration: 0159 Loss: 0.69778 Time: 0.00501
Iteration: 0160 Loss: 0.68547 Time: 0.00561
Iteration: 0161 Loss: 0.67819 Time: 0.00594
Iteration: 0162 Loss: 0.69174 Time: 0.00531
Iteration: 0163 Loss: 0.68517 Time: 0.00571
Iteration: 0164 Loss: 0.68628 Time: 0.00598
Iteration: 0165 Loss: 0.66981 Time: 0.00599
Iteration: 0166 Loss: 0.67856 Time: 0.00546
Iteration: 0167 Loss: 0.67616 Time: 0.00554
Iteration: 0168 Loss: 0.67418 Time: 0.00562
Iteration: 0169 Loss: 0.68462 Time: 0.00538
Iteration: 0170 Loss: 0.67660 Time: 0.00581
Iteration: 0171 Loss: 0.66608 Time: 0.00683
Iteration: 0172 Loss: 0.66874 Time: 0.00497
Iteration: 0173 Loss: 0.67101 Time: 0.00630
Iteration: 0174 Loss: 0.67149 Time: 0.00535
Iteration: 0175 Loss: 0.66881 Time: 0.00493
Iteration: 0176 Loss: 0.67147 Ti

Iteration: 0151 Loss: 0.70418 Time: 0.00706
Iteration: 0152 Loss: 0.68745 Time: 0.00822
Iteration: 0153 Loss: 0.70856 Time: 0.00511
Iteration: 0154 Loss: 0.69962 Time: 0.00689
Iteration: 0155 Loss: 0.69579 Time: 0.00626
Iteration: 0156 Loss: 0.69192 Time: 0.00474
Iteration: 0157 Loss: 0.68149 Time: 0.00632
Iteration: 0158 Loss: 0.69188 Time: 0.00578
Iteration: 0159 Loss: 0.67933 Time: 0.00800
Iteration: 0160 Loss: 0.68392 Time: 0.00701
Iteration: 0161 Loss: 0.68253 Time: 0.00600
Iteration: 0162 Loss: 0.68822 Time: 0.00800
Iteration: 0163 Loss: 0.67862 Time: 0.00544
Iteration: 0164 Loss: 0.67990 Time: 0.00599
Iteration: 0165 Loss: 0.67412 Time: 0.00654
Iteration: 0166 Loss: 0.67460 Time: 0.00517
Iteration: 0167 Loss: 0.67885 Time: 0.00630
Iteration: 0168 Loss: 0.67519 Time: 0.00655
Iteration: 0169 Loss: 0.68106 Time: 0.00605
Iteration: 0170 Loss: 0.66839 Time: 0.00588
Iteration: 0171 Loss: 0.67125 Time: 0.00534
Iteration: 0172 Loss: 0.68057 Time: 0.00566
Iteration: 0173 Loss: 0.66984 Ti

Iteration: 0122 Loss: 0.73827 Time: 0.00595
Iteration: 0123 Loss: 0.73205 Time: 0.00690
Iteration: 0124 Loss: 0.73817 Time: 0.00503
Iteration: 0125 Loss: 0.73291 Time: 0.00504
Iteration: 0126 Loss: 0.74415 Time: 0.00601
Iteration: 0127 Loss: 0.74098 Time: 0.00642
Iteration: 0128 Loss: 0.74693 Time: 0.00599
Iteration: 0129 Loss: 0.73150 Time: 0.00500
Iteration: 0130 Loss: 0.72410 Time: 0.00589
Iteration: 0131 Loss: 0.73005 Time: 0.00593
Iteration: 0132 Loss: 0.72417 Time: 0.00523
Iteration: 0133 Loss: 0.71848 Time: 0.00481
Iteration: 0134 Loss: 0.72144 Time: 0.00600
Iteration: 0135 Loss: 0.71250 Time: 0.00397
Iteration: 0136 Loss: 0.70355 Time: 0.00500
Iteration: 0137 Loss: 0.70379 Time: 0.00611
Iteration: 0138 Loss: 0.71395 Time: 0.00547
Iteration: 0139 Loss: 0.70917 Time: 0.00653
Iteration: 0140 Loss: 0.70760 Time: 0.00601
Iteration: 0141 Loss: 0.71688 Time: 0.00599
Iteration: 0142 Loss: 0.70727 Time: 0.00601
Iteration: 0143 Loss: 0.69108 Time: 0.00501
Iteration: 0144 Loss: 0.70778 Ti

...

Iteration: 0058 Loss: 0.97912 Time: 0.00593
Iteration: 0059 Loss: 0.94925 Time: 0.00600
Iteration: 0060 Loss: 0.95373 Time: 0.00601
Iteration: 0061 Loss: 0.94117 Time: 0.00599
Iteration: 0062 Loss: 0.94053 Time: 0.00600
Iteration: 0063 Loss: 0.95653 Time: 0.00555
Iteration: 0064 Loss: 0.94930 Time: 0.00664
Iteration: 0065 Loss: 0.93188 Time: 0.00584
Iteration: 0066 Loss: 0.94249 Time: 0.00598
Iteration: 0067 Loss: 0.92834 Time: 0.00608
Iteration: 0068 Loss: 0.91061 Time: 0.00624
Iteration: 0069 Loss: 0.93254 Time: 0.00569
Iteration: 0070 Loss: 0.93046 Time: 0.00599
Iteration: 0071 Loss: 0.91567 Time: 0.00558
Iteration: 0072 Loss: 0.91153 Time: 0.00601
Iteration: 0073 Loss: 0.91794 Time: 0.00631
Iteration: 0074 Loss: 0.91132 Time: 0.00564
Iteration: 0075 Loss: 0.89671 Time: 0.00610
Iteration: 0076 Loss: 0.89224 Time: 0.00491
Iteration: 0077 Loss: 0.89410 Time: 0.00600
Iteration: 0078 Loss: 0.87614 Time: 0.00520
Iteration: 0079 Loss: 0.86864 Time: 0.00580
Iteration: 0080 Loss: 0.86873 Ti

Iteration: 0035 Loss: 1.15725 Time: 0.00699
Iteration: 0036 Loss: 1.14616 Time: 0.00705
Iteration: 0037 Loss: 1.14095 Time: 0.00696
Iteration: 0038 Loss: 1.09630 Time: 0.00595
Iteration: 0039 Loss: 1.14526 Time: 0.00613
Iteration: 0040 Loss: 1.08820 Time: 0.00547
Iteration: 0041 Loss: 1.11487 Time: 0.00679
Iteration: 0042 Loss: 1.08900 Time: 0.00619
Iteration: 0043 Loss: 1.04229 Time: 0.00500
Iteration: 0044 Loss: 1.10107 Time: 0.00600
Iteration: 0045 Loss: 1.04805 Time: 0.00605
Iteration: 0046 Loss: 1.07622 Time: 0.00595
Iteration: 0047 Loss: 1.06323 Time: 0.00662
Iteration: 0048 Loss: 1.04604 Time: 0.00639
Iteration: 0049 Loss: 1.03346 Time: 0.00506
Iteration: 0050 Loss: 1.04997 Time: 0.00616
Iteration: 0051 Loss: 1.03402 Time: 0.00578
Iteration: 0052 Loss: 1.04222 Time: 0.00642
Iteration: 0053 Loss: 1.02331 Time: 0.00590
Iteration: 0054 Loss: 0.99554 Time: 0.00703
Iteration: 0055 Loss: 1.04001 Time: 0.00750
Iteration: 0056 Loss: 0.99865 Time: 0.00501
Iteration: 0057 Loss: 1.00062 Ti

Iteration: 0025 Loss: 1.25711 Time: 0.00800
Iteration: 0026 Loss: 1.24995 Time: 0.00599
Iteration: 0027 Loss: 1.22569 Time: 0.00700
Iteration: 0028 Loss: 1.22359 Time: 0.00800
Iteration: 0029 Loss: 1.24255 Time: 0.00701
Iteration: 0030 Loss: 1.19367 Time: 0.00699
Iteration: 0031 Loss: 1.21768 Time: 0.00700
Iteration: 0032 Loss: 1.15309 Time: 0.00710
Iteration: 0033 Loss: 1.15250 Time: 0.00637
Iteration: 0034 Loss: 1.19188 Time: 0.00764
Iteration: 0035 Loss: 1.11184 Time: 0.00553
Iteration: 0036 Loss: 1.14076 Time: 0.00748
Iteration: 0037 Loss: 1.12922 Time: 0.00601
Iteration: 0038 Loss: 1.14884 Time: 0.00599
Iteration: 0039 Loss: 1.13160 Time: 0.00720
Iteration: 0040 Loss: 1.09505 Time: 0.00581
Iteration: 0041 Loss: 1.09388 Time: 0.00699
Iteration: 0042 Loss: 1.09638 Time: 0.00700
Iteration: 0043 Loss: 1.11032 Time: 0.00617
Iteration: 0044 Loss: 1.07253 Time: 0.00783
Iteration: 0045 Loss: 1.06246 Time: 0.00601
Iteration: 0046 Loss: 1.06042 Time: 0.00800
Iteration: 0047 Loss: 1.04472 Ti

Done! Louvain has found 6 communities 

Setting up the model and the optimizer
Done! 

Preprocessing on message passing matrices
Done! 

Initializing TF session
Done! 

STEP 2/3 - MODEL TRAINING 

Starting training
Iteration: 0001 Loss: 1.65759 Time: 0.19001
Iteration: 0002 Loss: 1.70978 Time: 0.00801
Iteration: 0003 Loss: 1.69929 Time: 0.00600
Iteration: 0004 Loss: 1.66408 Time: 0.00700
Iteration: 0005 Loss: 1.54505 Time: 0.00600
Iteration: 0006 Loss: 1.60531 Time: 0.00800
Iteration: 0007 Loss: 1.55414 Time: 0.00599
Iteration: 0008 Loss: 1.59092 Time: 0.00701
Iteration: 0009 Loss: 1.51445 Time: 0.00600
Iteration: 0010 Loss: 1.58453 Time: 0.00600
Iteration: 0011 Loss: 1.48316 Time: 0.00700
Iteration: 0012 Loss: 1.51367 Time: 0.00794
Iteration: 0013 Loss: 1.40420 Time: 0.00700
Iteration: 0014 Loss: 1.43935 Time: 0.00795
Iteration: 0015 Loss: 1.38609 Time: 0.00801
Iteration: 0016 Loss: 1.34947 Time: 0.00824
Iteration: 0017 Loss: 1.41395 Time: 0.00800
Iteration: 0018 Loss: 1.36446 Time: 0

Iteration: 0194 Loss: 0.64910 Time: 0.00696
Iteration: 0195 Loss: 0.65790 Time: 0.00701
Iteration: 0196 Loss: 0.65913 Time: 0.00603
Iteration: 0197 Loss: 0.66074 Time: 0.00641
Iteration: 0198 Loss: 0.65196 Time: 0.00653
Iteration: 0199 Loss: 0.64725 Time: 0.00656
Iteration: 0200 Loss: 0.65863 Time: 0.00595
Done! 

STEP 3/3 - MODEL EVALUATION 

Computing the final embedding vectors, for evaluation
Done! 

Testing: link prediction
Done! 



In [6]:
# Report final results
print("FINAL RESULTS \n")

print('Recall: the selected task was "Task 2", i.e., joint community detection and link prediction, on', FLAGS.dataset)

print("All scores reported below are computed over the", FLAGS.nb_run, "run(s) \n")

print("Link prediction:\n")
print("Mean AUC score: ", np.mean(mean_roc))
print("Std of AUC scores: ", np.std(mean_roc), "\n")

print("Mean AP score: ", np.mean(mean_ap))
print("Std of AP scores: ", np.std(mean_ap), "\n \n")

FINAL RESULTS 

Recall: the selected task was "Task 2", i.e., joint community detection and link prediction, on celegans
All scores reported below are computed over the 10 run(s) 

Link prediction:

Mean AUC score:  0.8470477770984365
Std of AUC scores:  0.016257558618347212 

Mean AP score:  0.8339304731182088
Std of AP scores:  0.01873121168411811 
 

